In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


In [2]:
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)

In [3]:
df = pd.read_csv('mushrooms.csv')


In [4]:
df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


In [5]:
print(f"class data : {df['class'].unique()}")
print(f"{df['class'].value_counts()}")
print(f"sample data : {df.shape[0]}, feature data : {df.shape[1]}")


class data : ['p' 'e']
class
e    4208
p    3916
Name: count, dtype: int64
sample data : 8124, feature data : 23


## Visialize Data

In [6]:
label_encoder = LabelEncoder()
df = df.apply(lambda x: label_encoder.fit_transform(x) if x.dtype == 'O' else x)

## Train test split

In [7]:
def custom_train_test_split(df, test_size, random_state=seed_value):
    train = df.sample(frac=(1 - test_size), random_state=random_state)
    test = df.drop(train.index)
    
    return train, test

## SVM classifier

In [8]:
class SVM_classifier():
    def __init__(self, learning_rate=0.001, lambda_parameter=0.01, iterations=1000):
        self.learning_rate = learning_rate
        self.lambda_parameter = lambda_parameter
        self.iterations = iterations
        self.weights = None
        self.bias = None
        
    def fit(self, X, Y):
        n_samples, n_features = X.shape
        
        # Initialize parameters
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        # Gradient Descent
        for _ in range(self.iterations):
            # Compute gradient
            svm_output = np.dot(X, self.weights) + self.bias
            hinge_loss = np.maximum(0, 1 - Y * svm_output)
            dW = np.dot(X.T, (-Y * hinge_loss)) / n_samples
            db = np.sum(-Y * hinge_loss) / n_samples
            
            # Update weights
            self.weights -= self.learning_rate * (dW + 2 * self.lambda_parameter * self.weights)
            self.bias -= self.learning_rate * db
            
    def predict(self, X):
        svm_output = np.dot(X, self.weights) + self.bias
        return np.sign(svm_output)



class SoftSVM_classifier():
    def __init__(self, learning_rate=0.001, iterations=1000, C=1.0, lamda=0):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.weights = None
        self.bias = None
        self.C = C
        self.lamda = 0
    
    def fit(self, X, Y):
        n_samples, n_features = X.shape
        
        # Initialize parameters
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        # Gradient Descent
        for iter in range(self.iterations):     
            
            # Compute gradient
            pred = np.dot(X, self.weights) + self.bias
            hinge_loss = np.maximum(0, 1-self.C-pred*Y)
            
            mask = (hinge_loss > 0 + self.C)
            dW = -np.dot(mask * Y, X) / len(X) + self.lamda * self.weights * 2
            dB = -np.sum(mask * Y) / len(X)
            
            # Update weights
            self.weights -= self.learning_rate * dW
            self.bias -= self.learning_rate * dB
        
    def predict(self, X):
        predictions = np.sign(np.dot(X, self.weights) + self.bias)
        return predictions
    

## Set up data and training model

In [9]:
full_train, test = custom_train_test_split(df, test_size=0.2)
train, validation = custom_train_test_split(full_train, test_size=0.2)

x_train = train.drop(['class'], axis=True)
y_train = train['class']

x_vali = validation.drop(['class'], axis=True)
y_vali = validation['class']

x_test = test.drop(['class'], axis=True)
y_test = test['class']

In [10]:
model_SVM = SVC()
model_customSVM = SVM_classifier()
model_softSVM = SoftSVM_classifier()

model_SVM.fit(x_train, y_train)
model_customSVM.fit(x_train, y_train)
model_softSVM.fit(x_train, y_train)

y_pred_SVM = model_SVM.predict(x_test)
y_pred_customSVM = model_customSVM.predict(x_test)
y_pred_softSVM = model_softSVM.predict(x_test)

## Performance

In [11]:
accuracy_model_SVM = np.mean(y_pred_SVM == y_test)
accuracy_model_customSVM = np.mean(y_pred_customSVM == y_test)
accuracy_model_softSVM = np.mean(y_pred_softSVM == y_test)

print(f"Accuracy SVM: {accuracy_model_SVM}")
print(f"Accuracy custom SVM: {accuracy_model_customSVM}")
print(f"Accuracy soft SVM: {accuracy_model_softSVM}")

Accuracy SVM: 0.9895384615384616
Accuracy custom SVM: 0.4787692307692308
Accuracy soft SVM: 0.5212307692307693
